In [1]:
import pandas as pd

In [2]:
cd ..

/home/evan/Documents/github/Feedlot


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
# load trades, orders, settlements, and tokens csv files
trades = pd.read_csv('data/cow_raw/trades.csv')
orders = pd.read_csv('data/cow_raw/orders.csv', low_memory=False)
settlements = pd.read_csv('data/cow_raw/settlements.csv')
tokens = pd.read_csv('data/cow_raw/tokens.csv')
solvers = pd.read_csv('data/transformed/solvers.csv')

In [5]:
# print all df column names
print(f'trades: {trades.columns}')
print(f'\norders: {orders.columns}')
print(f'\nsettlements: {settlements.columns}')
print(f'\ntokens: {tokens.columns}')
print(f'\nsolvers: {solvers.columns}')

trades: Index(['Unnamed: 0', 'id', 'timestamp', 'gasPrice', 'feeAmount', 'txHash',
       'settlement', 'buyAmount', 'sellAmount', 'sellToken', 'buyToken',
       'order', 'buyAmountEth', 'sellAmountEth', 'buyAmountUsd',
       'sellAmountUsd'],
      dtype='object')

orders: Index(['Unnamed: 0', 'id', 'owner', 'tradesTimestamp', 'invalidateTimestamp',
       'presignTimestamp', 'trades', 'isSigned'],
      dtype='object')

settlements: Index(['Unnamed: 0', 'id', 'txHash', 'firstTradeTimestamp', 'trades'], dtype='object')

tokens: Index(['Unnamed: 0', 'id', 'address', 'firstTradeTimestamp', 'name', 'symbol',
       'decimals', 'totalVolume', 'priceEth', 'priceUsd', 'history',
       'hourlyTotals', 'dailyTotals'],
      dtype='object')

solvers: Index(['address', 'environment', 'name', 'active'], dtype='object')


In [6]:
def removeCol(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Removes a column from a dataframe if it exists. Return the DataFrame
    """
    if col_name in df.columns:
        df.drop(col_name, axis=1, inplace=True)
        print(f'Removed {col_name}')
    return df

In [7]:
# package dfs into a dictionary for automation
dfs = {'trades': trades, 'orders': orders, 'settlements': settlements, 'tokens': tokens, 'solvers': solvers}

In [8]:
# remove columns that are not needed
for df_name, df in dfs.items():
    df = removeCol(df, 'Unnamed: 0')
    df_name = df

Removed Unnamed: 0
Removed Unnamed: 0
Removed Unnamed: 0
Removed Unnamed: 0


In [9]:
# scan every column in dfs and return a list of columns that have the value 0xf2d21ad3c88170d4ae52bbbeba80cb6078d276f4
def findColVal(df: pd.DataFrame, value: str) -> list:
    """
    Scans every column in a DataFrame and returns a list of columns that have the value
    """
    cols = []
    for col in df.columns:
        if value in df[col].values:
            cols.append(col)
    return cols

In [10]:
for df_name, df in dfs.items():
    print(f'{df_name}')
    print(findColVal(df, '0xf2d21ad3c88170d4ae52bbbeba80cb6078d276f4')) 

trades
[]
orders
[]
settlements
['trades']
tokens
[]
solvers
['address']


/tmp/ipykernel_416054/1392819561.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if value in df[col].values:


In [11]:
# inner join settlements and solvers on settlements trades column and solvers address column
settlements_merged = settlements.merge(solvers, left_on='trades', right_on='address', how='inner')

In [12]:
settlements

,id,txHash,firstTradeTimestamp,trades
0,0x000012606964c50c94724a64bfc15f9a7746c6acb4923197575ccd94b23d2db9,0x000012606964c50c94724a64bfc15f9a7746c6acb4923197575ccd94b23d2db9,1648449982,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718
1,0x00008e5e87877e38a3855e208d61d5d859ff6b2895be7cd358b3c121b10ec807,0x00008e5e87877e38a3855e208d61d5d859ff6b2895be7cd358b3c121b10ec807,1670299751,0xa21740833858985e4d801533a808786d3647fb83
2,0x000098565f5db850d433551d27993087f2fbf4ec533f7fdd8fd868302caca19e,0x000098565f5db850d433551d27993087f2fbf4ec533f7fdd8fd868302caca19e,1656651460,0xc9ec550bea1c64d779124b23a26292cc223327b6
3,0x0000b671e285b4825e8205f9ff206baee7849d9331bf812674de83fd4f8fca4e,0x0000b671e285b4825e8205f9ff206baee7849d9331bf812674de83fd4f8fca4e,1648145708,0xdae69affe582d36f330ee1145995a53fab670962
4,0x0000eb0ede5f863bdb1716ee2614e8002b798004704a47b758d75f5f86f9838f,0x0000eb0ede5f863bdb1716ee2614e8002b798004704a47b758d75f5f86f9838f,1643815289,0xde786877a10dbb7eba25a4da65aecf47654f08ab
...,...,...,...,...
313899,0xffff448378046f5194db5cd4e403727017f0513cf382dfd83ac67981aceffc9b,0xffff448378046f5194db5cd4e403727017f0513cf382dfd83ac67981aceffc9b,1631456381,0xa6ddbd0de6b310819b49f680f65871bee85f517e
313900,0xffff5b27631067de3ee5c801d49b618667e853cb9a74cab0bf95499436efd37b,0xffff5b27631067de3ee5c801d49b618667e853cb9a74cab0bf95499436efd37b,1647322829,0xe92f359e6f05564849afa933ce8f62b8007a1d5d
313901,0xffff8c9d85e109a627b912c61696a86d8931d8640a678ad2da41475aa81fb4d9,0xffff8c9d85e109a627b912c61696a86d8931d8640a678ad2da41475aa81fb4d9,1672066463,0xc9ec550bea1c64d779124b23a26292cc223327b6
313902,0xffffa109fe6f444be6ab42c6704214a4c820d1878fcb74ff8ad30148e90c4211,0xffffa109fe6f444be6ab42c6704214a4c820d1878fcb74ff8ad30148e90c4211,1671801527,0xa21740833858985e4d801533a808786d3647fb83


In [13]:
# rename trades to solvers and drop address column
settlements_merged.rename(columns={'trades': 'solvers'}, inplace=True)
settlements_merged.drop('address', axis=1, inplace=True)

In [14]:
settlements_merged

,id,txHash,firstTradeTimestamp,solvers,environment,name,active
0,0x000012606964c50c94724a64bfc15f9a7746c6acb4923197575ccd94b23d2db9,0x000012606964c50c94724a64bfc15f9a7746c6acb4923197575ccd94b23d2db9,1648449982,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False
1,0x0003aa64ac253d7314f21d62f300c01b64d0bef042665ce09dac19a6c6a5ccd0,0x0003aa64ac253d7314f21d62f300c01b64d0bef042665ce09dac19a6c6a5ccd0,1649603726,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False
2,0x000479b81b2b164290f1624f24ee3c79b57cb1bc1554f03f034b47cdf76d74a8,0x000479b81b2b164290f1624f24ee3c79b57cb1bc1554f03f034b47cdf76d74a8,1651855777,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False
3,0x00057afeeecde2934287b2b28f32601b575143ca43f23cff979aa970d68dacd8,0x00057afeeecde2934287b2b28f32601b575143ca43f23cff979aa970d68dacd8,1652449728,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False
4,0x0007a5a7a3575217d5ee457c8d98db2b65882704a60c0bf5bbaf229d1c4a0b4d,0x0007a5a7a3575217d5ee457c8d98db2b65882704a60c0bf5bbaf229d1c4a0b4d,1648451608,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False
...,...,...,...,...,...,...,...
313899,0x73371855eea7cd99182ff4f35e46f44c4f0cafa70f540aa40d8cfb75f8b0bf08,0x73371855eea7cd99182ff4f35e46f44c4f0cafa70f540aa40d8cfb75f8b0bf08,1636644190,0x2caef7f0ee82fb0abf1ab0dcd3a093803002e705,test,Test Solver 1,True
313900,0xa570709694952ac8bff15ab360f3fe15017616d45734682f64e2ecd85c74ad01,0xa570709694952ac8bff15ab360f3fe15017616d45734682f64e2ecd85c74ad01,1633696088,0x2caef7f0ee82fb0abf1ab0dcd3a093803002e705,test,Test Solver 1,True
313901,0xb88c853cda8ff918e7e5a4b0a5c414ae0387f2fabbd0ac0635141afc94bcb387,0xb88c853cda8ff918e7e5a4b0a5c414ae0387f2fabbd0ac0635141afc94bcb387,1633695675,0x2caef7f0ee82fb0abf1ab0dcd3a093803002e705,test,Test Solver 1,True
313902,0xc1a4a9171783bc7ca22f2bf90cdefd4ced7a805c1a29c87e795c509d2c784ecf,0xc1a4a9171783bc7ca22f2bf90cdefd4ced7a805c1a29c87e795c509d2c784ecf,1636644117,0x2caef7f0ee82fb0abf1ab0dcd3a093803002e705,test,Test Solver 1,True


In [15]:
# add settlements_merged to settlement dictionary key
dfs['settlements'] = settlements_merged

In [16]:
# which dataframes have txHash column?
for df_name, df in dfs.items():
    if 'txHash' in df.columns:
        print(df_name)
    

trades
settlements


In [17]:
# inner merge settlements on trades txHash
trades_merged = trades.merge(settlements_merged, left_on='txHash', right_on='txHash', how='inner')

In [18]:
trades_merged

,id_x,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,sellToken,buyToken,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,id_y,firstTradeTimestamp,solvers,environment,name,active
0,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0|0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf|59,1663907363,6062637482,1344622,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,917011543008284856417,4093394367,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0,3.047480,3.049075,4087.560206,4093.394367,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,1663907363,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True
1,0x0000543dbe9253542d4c3020c5ed40fefd3520dcf5ea9c16e71d611031629c50a85f11d4d6119848a0d2b2b8e270a6846383f39f6199100a|0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af|1,1637419389,96392802490,7508432409899440,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,20887903,5403161965640052848,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x21ad647b8f4fe333212e735bfc1f36b4941e6ad2,0x0000543dbe9253542d4c3020c5ed40fefd3520dcf5ea9c16e71d611031629c50a85f11d4d6119848a0d2b2b8e270a6846383f39f6199100a,0.000000,5.403162,0.000000,23288.207524,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,1637419389,0xe92f359e6f05564849afa933ce8f62b8007a1d5d,prod,Gnosis_0x,False
2,0x02e33692094dfc30ea6f0e45d6f177fe944530736504023bac5c05f92f683d76b97c34053e0088d406b1f154e65fe27962c41f4161990fda|0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af|1,1637419389,96392802490,31682555,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,50905988578,50963592006,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xdac17f958d2ee523a2206206994597c13d831ec7,0x02e33692094dfc30ea6f0e45d6f177fe944530736504023bac5c05f92f683d76b97c34053e0088d406b1f154e65fe27962c41f4161990fda,11.810840,11.824205,50905.988578,50963.592006,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,1637419389,0xe92f359e6f05564849afa933ce8f62b8007a1d5d,prod,Gnosis_0x,False
3,0x9d7c5a559fdb544faf18e94cf5b080753727ffaf9b5625bc59806c34195e80528c6d571fcfbc01d5258655310041396f3b02ce2961990dbb|0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af|1,1637419389,96392802490,48290256,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,68704617924013822452,2000000000,0xdac17f958d2ee523a2206206994597c13d831ec7,0x4e352cf164e64adcbad318c3a1e222e9eba4ce42,0x9d7c5a559fdb544faf18e94cf5b080753727ffaf9b5625bc59806c34195e80528c6d571fcfbc01d5258655310041396f3b02ce2961990dbb,0.449878,0.464026,1943.547436,2000.000000,0x8ed73664124b6683be7064af365ce972679b94964680ccb79056ed5192e1a9af,1637419389,0xe92f359e6f05564849afa933ce8f62b8007a1d5d,prod,Gnosis_0x,False
4,0x00006f4028421299b3bf398dace422a6230beca51223a47b407ad0038351bd07a4d0b8a5cf2fcb72abec58bf9be156a2e0a58b7e62555649|0x1759c9e55bed47c03ca02f78a63ed674815d35ee971cd81d0733737cdd528257|3,1649758670,37462722322,231083551837426720,0x1759c9e55bed47c03ca02f78a63ed674815d35ee971cd81d0733737cdd528257,0x1759c9e55bed47c03ca02f78a63ed674815d35ee971cd81d0733737cdd528257,83488515439092675372,50512108780891254932,0x0d438f3b5175bebc262bf23753c1e53d03432bde,0xba100000625a3754423978a60c9317c58a424e3d,0x00006f4028421299b3bf398dace422a6230beca51223a47b407ad0038351bd07a4d0b8a5cf2fcb72abec58bf9be156a2e0a58b7e62555649,0.000000,0.431026,0.000000,1301.451984,0x1759c9e55bed47c03ca02f78a63ed674815d35ee971cd81d0733737cdd528257,1649758670

In [19]:
# drop id_y from trades_merged because it is the same as the settlement column
trades_merged.drop('id_y', axis=1, inplace=True)

In [20]:
tokens_lim = dfs['tokens'][['address', 'symbol', 'decimals']]

In [21]:
# merge tokens_lim on trades_merged sellToken with address
trades_merged_sell = trades_merged.merge(tokens_lim, left_on='sellToken', right_on='address', how='inner')
# merge tokens_lim on trades_merged buyToken with address
cow_swaps = trades_merged_sell.merge(tokens_lim, left_on='buyToken', right_on='address', how='inner')

In [22]:
cow_swaps

,id_x,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,sellToken,buyToken,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,firstTradeTimestamp,solvers,environment,name,active,address_x,symbol_x,decimals_x,address_y,symbol_y,decimals_y
0,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0|0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf|59,1663907363,6062637482,1344622,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,917011543008284856417,4093394367,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0,3.047480,3.049075,4087.560206,4093.394367,1663907363,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,FXS,18
1,0x0004d9673bd844cc4d44a33b1f5c1df7ef6cf5eff63ccaaf99d4442089a2deb0d1f2739ad714045be6146915275d0a2b822ec1cc636c5b20|0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4|4,1668043871,30744415450,37223401,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,8621106771373760291720,45331499213,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x0004d9673bd844cc4d44a33b1f5c1df7ef6cf5eff63ccaaf99d4442089a2deb0d1f2739ad714045be6146915275d0a2b822ec1cc636c5b20,39.399436,39.811826,44861.934154,45331.499213,1668043871,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,FXS,18
2,0x00668e787a470a68eee83463fb4618ce9bedec7c50ac7ea9d2bf0d073792fe4d95861c41e7767dc737bca90b175ad51e5f7b9ada637cde92|0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32|150,1669126187,24494377559,13386244,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,1583112031078343925188,6719000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x00668e787a470a68eee83463fb4618ce9bedec7c50ac7ea9d2bf0d073792fe4d95861c41e7767dc737bca90b175ad51e5f7b9ada637cde92,5.996258,5.917571,6821.422376,6719.000000,1669126187,0xb20b86c4e6deeb432a22d773a221898bbbd03036,prod,Gnosis_1inch,True,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,FXS,18
3,0x00a3de84225174293e86edf848100dfb1123ea727209aa63bd9d72edc9838de8254725a2d9989ab4202e1f6a6560b970dc4a67da6310958d|0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982|151,1662029515,12869344497,4808912,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,369820837360497071942,2138328191,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x00a3de84225174293e86edf848100dfb1123ea727209aa63bd9d72edc9838de8254725a2d9989ab4202e1f6a6560b970dc4a67da6310958d,1.380130,1.372932,2126.884281,2138.328191,1662029515,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,FXS,18
4,0x00a5ee906f05258014318b5488bded192c21ed6a9ee7a65506a80cb459d362d8cbdd2638b5f2e0360ea43806936276ba16d0a4ab6289a3b9|0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8|1,1653185840,14600197606,20683162,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,8901647727774327086836,59366191057,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x00a5ee906f05258014318b5488bded192c21ed6a9ee7a65506a80cb

In [23]:
# drop sellToken and then rename symbol_x to sellTokenSymbol and decimals_x to sellTokenDecimals
cow_swaps.drop('sellToken', axis=1, inplace=True)
cow_swaps.rename(columns={'symbol_x': 'sellTokenSymbol', 'decimals_x': 'sellTokenDecimals'}, inplace=True)

In [24]:
# drop buyToken and then rename symbol_y to buyTokenSymbol and decimals_y to buyTokenDecimals
cow_swaps.drop('buyToken', axis=1, inplace=True)
cow_swaps.rename(columns={'symbol_y': 'buyTokenSymbol', 'decimals_y': 'buyTokenDecimals'}, inplace=True)

In [25]:
#drop address_x, address_y, firstTradeTimestamp, id_x
cow_swaps.drop(['address_x', 'address_y', 'firstTradeTimestamp', 'id_x'], axis=1, inplace=True)

In [26]:
# drop row if buyAmountUsd is 0
cow_swaps = cow_swaps[cow_swaps['buyAmountUsd'] != 0]

# drop row if sellAmountUsd is 0
cow_swaps = cow_swaps[cow_swaps['sellAmountUsd'] != 0]

In [28]:
# saved cow_swaps to csv
cow_swaps.to_csv('data/transformed/cow_swaps.csv', index=False)

In [29]:
cow_swaps.columns

Index(['timestamp', 'gasPrice', 'feeAmount', 'txHash', 'settlement',
       'buyAmount', 'sellAmount', 'order', 'buyAmountEth', 'sellAmountEth',
       'buyAmountUsd', 'sellAmountUsd', 'solvers', 'environment', 'name',
       'active', 'sellTokenSymbol', 'sellTokenDecimals', 'buyTokenSymbol',
       'buyTokenDecimals'],
      dtype='object')

In [30]:
cow_swaps

,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals
0,1663907363,6062637482,1344622,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,917011543008284856417,4093394367,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0,3.047480,3.049075,4087.560206,4093.394367,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
1,1668043871,30744415450,37223401,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,8621106771373760291720,45331499213,0x0004d9673bd844cc4d44a33b1f5c1df7ef6cf5eff63ccaaf99d4442089a2deb0d1f2739ad714045be6146915275d0a2b822ec1cc636c5b20,39.399436,39.811826,44861.934154,45331.499213,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
2,1669126187,24494377559,13386244,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,1583112031078343925188,6719000000,0x00668e787a470a68eee83463fb4618ce9bedec7c50ac7ea9d2bf0d073792fe4d95861c41e7767dc737bca90b175ad51e5f7b9ada637cde92,5.996258,5.917571,6821.422376,6719.000000,0xb20b86c4e6deeb432a22d773a221898bbbd03036,prod,Gnosis_1inch,True,USDC,6,FXS,18
3,1662029515,12869344497,4808912,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,369820837360497071942,2138328191,0x00a3de84225174293e86edf848100dfb1123ea727209aa63bd9d72edc9838de8254725a2d9989ab4202e1f6a6560b970dc4a67da6310958d,1.380130,1.372932,2126.884281,2138.328191,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
4,1653185840,14600197606,20683162,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,8901647727774327086836,59366191057,0x00a5ee906f05258014318b5488bded192c21ed6a9ee7a65506a80cb459d362d8cbdd2638b5f2e0360ea43806936276ba16d0a4ab6289a3b9,29.458128,30.052786,58191.504954,59366.191057,0x6fa201c3aff9f1e4897ed14c7326cf27548d9c35,prod,Otex,False,USDC,6,FXS,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434224,1670764415,12178478529,377900329134939712,0x1f4f1f738cdcef3ee5ad99afda6873aab5bdd42e5a7ec97fce260fffed3c9d68,0x1f4f1f738cdcef3ee5ad99afda6873aab5bdd42e5a7ec97fce260fffed3c9d68,15589999145475962,34985792278947154589,0x7c2fb6ce9efd7a527d2ad16cb4e73ecca41cb1747f902a5866d4e664a9920365d038026c1ec42757e1f4b45bfd422c3462ed4f6c6395de42,0.353634,3.438601,450.586996,4537.000529,0xda869be4adea17ad39e1dfece1bc92c02491504f,prod,Gnosis_0x,True,UwU,18,wMEMO,18
434228,1648717440,31172516953,15819157520127830016,0xc5ca786176605a439c84693f9ab18c0218bc675060d179f39eb1b1c186fa8d91,0xc5ca786176605a439c84693f9ab18c0218bc675060d179f39eb1b1c186fa8d91,7737731671073522318726,4018989407490731756605,0xb213d396c81d5357be4f4c680d84fe8c8028fbf3b0dba1ea78a0be92d06646a437ad67621aad50871d2cfa938e9804ea81329a9e62456f2b,2.277529,2.286410,7750.205750,7780.426291,0xde1c59bc25d806ad9ddcbe246c4b5e5505645718,prod,Gnosis_1inch,False,ENJ,18,PUNDIX,18
434232,1640522370,40680086403,706476638153923968,0x89e1d5e91671cc612c36ab99b4bf6ce47465f70b71500f01110e3d19b0b25821,0x89e1d5e91671cc612c36ab99b4bf6ce47465f70b71500f01110e3d19b0b25821,1425076824016,11180859049544539990,0x4baab5aa92068042fea1b84286fd5d26df42831bd411726b1e5361f442c2da2618fc0099cc649f53825f60fdda5f10f68965ed0f61c868fb,0.043301,0.044343,174.445209,178.641772,0xe92f359e6f05564849afa933ce8f62b8007a1d5d,prod,Gnosis_0x,False,GTC,18,FUN,8
434239,1672408487,15114360666,36533260155803959296,0xbf681530f33464211f7b206361b6281681906ad7ab63322906435d0f9fa4d690,0xbf681530f33464211f7b2